## <center>利用 RNN 训练语言模型</center>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
gpu_no = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_no
# 定义TensorFlow配置
config = tf.ConfigProto()
# 配置GPU内存分配方式，按需增长，很关键
config.gpu_options.allow_growth = True
# 配置可使用的显存比例
config.gpu_options.per_process_gpu_memory_fraction = 0.3
# 在创建session的时候把config作为参数传进去
sess = tf.Session(config = config)

### 1 定义基本工具函数

In [2]:
import random
import time
from collections import Counter
start_time = time.time()

def elapsed(sec):
    if sec < 60:
        return str(sec) + " sec"
    elif sec < 60*60:
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + "h"
tf.reset_default_graph()
training_file = './data/text/word.txt'

# 处理多个中文文件
def read_all_txt(txt_files):
    words = []
    for txt_file in txt_files:
        file_words = get_ch_words(txt_file)
        words.append(file_words)
    return words
# 处理汉字
def get_ch_words(txt_file):
    words = ""
    with open(txt_file,'rb') as f:
        for word in f:
            words = words + word.decode('utf-8')
    return words

# 将文件里面的字符转换成向量
def get_ch_words_v(txt_file,word_num_map,txt_words=None):
    words_size = len(word_num_map)
    to_num = lambda word:word_num_map.get(word,words_size)
    if txt_file != None:
        txt_words = get_ch_words(txt_file)
    txt_words_vector = list(map(to_num,txt_words))
    return txt_words_vector

### 2 样本预处理

In [3]:
training_data = get_ch_words(training_file)
print(training_data)

这种感觉，愈发地受到强烈震撼，简直超乎了所有想象，就是要将人生之花开出更加丰硕、更加红艳花朵，在三生三世，遍溢余香，掌声雷动，经久不息。浅黄与明黄扉页之上，红红枫叶蹁跹舞蹈，烘焙“枫叶正红”四字，别开生面地飘逸曹树清老先生书集封面，仿佛将曹老神韵，清瘦硬朗，适中身材，熠熠有神眼眸，荡漾精气神十足，仙风道骨，鹤然峭立，把枯藤老树，融化殆尽。使我在拜读他之书作，更是与他心灵相通，灵魂嫁接，老树新花，怒放璀璨绚丽。


In [4]:
couter = Counter(training_data)
words = sorted(couter)
words_size = len(words)
word_num_map = dict(zip(words,range(words_size)))
print("word_num_map:")
print(word_num_map)
print("word list size:")
print(words_size)
word_v = get_ch_words_v(training_file,word_num_map)
print(word_v)

word_num_map:
{'仙': 19, '绚': 106, '飘': 137, '熠': 88, '直': 93, '身': 123, '香': 138, '璀': 89, '放': 63, '峭': 46, '有': 69, '四': 36, '书': 15, '殆': 77, '掌': 60, '受': 34, '中': 9, '尽': 45, '息': 51, '红': 104, '灵': 83, '藤': 112, '超': 118, '封': 42, '乎': 14, '佛': 22, '所': 56, '了': 16, '生': 91, '人': 17, '将': 43, '动': 30, '跹': 120, '余': 21, '这': 124, '璨': 90, '感': 54, '我': 55, '鹤': 141, '漾': 82, '然': 87, '荡': 111, '舞': 108, '怒': 50, '丽': 11, '遍': 128, '页': 135, '风': 136, '立': 101, '心': 49, '蹈': 122, '要': 114, '道': 129, '接': 61, '加': 29, '适': 125, '正': 76, '别': 27, '拜': 59, '朗': 70, '融': 113, '”': 1, '三': 4, '就': 44, '愈': 53, '、': 2, '材': 72, '强': 48, '嫁': 40, '字': 41, '浅': 79, '上': 5, '十': 32, '枯': 74, '仿': 20, '与': 7, '面': 133, '声': 39, '韵': 134, '地': 38, '气': 78, '种': 100, '丰': 10, '简': 102, '树': 75, '硬': 98, '枫': 73, '蹁': 121, '集': 130, '通': 126, '雷': 131, '想': 52, '艳': 109, '神': 99, '发': 33, '觉': 115, '曹': 68, '化': 31, '骨': 139, '明': 65, '震': 132, '逸': 127, '眸': 95, '作': 23, '眼': 96, '经': 105, '把

### 3 构建模型

本例中,使用多层 RNN 模型,后面接一个 softmax 分类, 对下一个字属于哪个向量进行分类,这里认为一个字就是一类.

#### (1) 设置参数定义占位符
学习率设置为 0.001, 迭代次数为 10000 次, 每 1000 次输出一次中间状态. 每次输入 4 个字,来预测第 5 个字. 网络模型使用了 3 层的 LSTM RNN, 第一层为 256 个 cell, 第二层和第三层都是 512 个 cell

In [5]:
# 定义参数
learning_rate = 0.001
training_iters = 10000
display_step = 1000
n_input = 4
n_hidden_1 = 256
n_hidden_2 = 512
n_hidden_3 = 512
# 定义占位符,其中,x 代表输入的 4 个连续文字,y 代表一个字,由于使用字索引向量的 one_hot 编码,
# 所以其大小为 word_size, 代表总共的字数,输入为每个字的 id, 对应的标签是 one_hot 编码.
x = tf.placeholder(dtype=tf.float32,shape=[None,n_input,1])
y = tf.placeholder(dtype=tf.float32,shape=[None,words_size])

#### (2) 定义网络结构

将 x 形状变换并按照时间序列拆分,然后放入 3 层 LSTM 网络,最终通过一个全连接生成 words_size 个节点,为后面的 softmax 做准备.

In [6]:
# 将输入数据变成 [batch_size,timesteps]
x_1 = tf.reshape(x,[-1,n_input])
# 将输入数据变成 [timesteps,batch_size] 
x_2 = tf.split(x_1,n_input,axis=1)
# 2-layer LSTM, 每层有 n_hidden 个 units
cell_1 = tf.nn.rnn_cell.LSTMCell(num_units=n_hidden_1)
cell_2 = tf.nn.rnn_cell.LSTMCell(num_units=n_hidden_2)
cell_3 = tf.nn.rnn_cell.LSTMCell(num_units=n_hidden_3)
# 构建多层 RNN 网络
rnn_cell = tf.nn.rnn_cell.MultiRNNCell([cell_1,cell_2,cell_3])
# 在 static_rnn 网络输入数据的格式必须是 timesteps 优先
outputs,states = tf.nn.static_rnn(rnn_cell,x_2,dtype=tf.float32)
# 预测的时候,使用全连接层,这一层的神经单元数量与 one_hot 位数相同
pred = tf.contrib.layers.fully_connected(outputs[-1],words_size,activation_fn=None)

#### (3) 定义优化器

使用 AdamOptimizer 优化器, loss 使用的是 softmax 的交叉熵,正确率是统计 one_hot 中索引对应的位置相同的个数

In [7]:
# 定义 loss 和 优化器
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(loss)

# 模型评估
correct_pred = tf.equal(tf.argmax(pred,axis=1),tf.argmax(y,axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,dtype=tf.float32))

### 4 训练模型

在训练过程中添加检查点功能,在 session 中每次随机取一个偏移量,然后取后面的4个文字向量当做输入,第五个文字向量当做标签用来计算 loss

In [8]:
savedir = "log/11_rnn_model/"
saver = tf.train.Saver(max_to_keep=1)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0.0
    loss_total = 0.0
    kpt = tf.train.latest_checkpoint(savedir)
    start_step = 0
    if kpt!=None:
        saver.restore(sess,kpt)
        index = kpt.find("-")
        start_step = int(kpt[index+1:])
        print(start_step)
        step = start_step
    while step < training_iters:
        # 随机取一个偏移位置
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0,n_input+1)
        input_words = [[word_label[i]] for i in range(offset,offset+n_input)]
        input_words = np.reshape(np.array(input_words),[-1,n_input,1])
        out_onehot = np.zeros([words_size],dtype=np.float32)
        out_onehot[word_label[offset+n_input]] = 1.0
        out_onehot = np.reshape(out_onehot,[1,-1])
        _,acc,lossval,onehot_pred = sess.run([optimizer,accuracy,loss,pred],feed_dict={x:input_words,y:out_onehot})
        loss_total += lossval
        if (step+1) % display_step == 0:
            print("iter= " + str(step+1) + ", average loss = "+ str(loss_total/display_step)+ ", average acc = " + str(100*acc))
            acc_total = 0.0
            loss_total = 0.0
            input_words_2 = [[word_label[i]] for i in range(offset,offset+n_input)]
            out_2 = words[word_label[word_label[offset + n_input]]]
            out_pred = words[int(tf.argmax(onehot_pred,axis=1).eval())]
            print("%s - [%s] vs [%s]" % (input_words_2,out_2,out_pred))
            saver.save(sess,savedir + "rnn_word.cpkt",global_step = step)
        step += 1
        offset += n_input + 1
    print("Finished!")
    saver.save(sess,savedir + "rnn_word.cpkt",global_step = step)
    print("elapsed time: ",elapsed(time.time()-start_time))

INFO:tensorflow:Restoring parameters from log/11_rnn_model/rnn_word.cpkt-10000
10000
Finished!
elapsed time:  2.482757568359375 sec
